# Tropical Cyclone Tracks and Characteristics Notebook Tutorial

This tutorial aims at giving a better idea abouth the operations streamflow behind the visualization of data in the first section of **TropiDash**. This section focuses on displaying information regarding the cyclone tracks and their spatial characteristics. More in particular the data available for consultation are:

1. Ensemble tracks forecast of the cyclone
2. Average forecast track of the cyclone 
3. Observed track of the cyclone
4. Strike Probability Map

As example for the tutorial we adopted the forecast of cyclone **LEE** on **September 7, 2023**. 

## Libraries Import

In [2]:
import pdbufr
import math
import ipyleaflet
import rasterio
import pandas as pd
import numpy as np
import ipywidgets as widgets
import branca.colormap as bc

from datetime import datetime, timedelta
from localtileserver import get_leaflet_tile_layer, TileClient
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

## Data Donwload

Since this is a tutorial, the cell below is is a raw cell instead of a code cell. The data have been already downloaded and you do not need to run it. As source of data for the forecast we use ECMWF open dataset, for the observed track we use __[IBTrACS](https://www.ncei.noaa.gov/products/international-best-track-archive)__.

Be aware that the code reported donwloads the latest data availabe for active cyclones from IBTrACS. If you run it it will overwrite the .csv file containing information about LEE that were availabe on the 7th September 2023. 

## Dataset Load and Pre-formatting

The forecast tracks data of ECMWF are in .bufr format to load them as a pandas dataframe we use the bufr library.

For the way the file are built is not possible to load together the column of the mean sea level pressure at the core of the cyclone and the column of the maximum wind speed at 10 meters within the cyclone system. Two different dataframes are created and then merged together. Also, in the original file the column of temporal information, i.e. the hours after the forecast date, might contain erros so we manually computed it and insert it in the dataframe. At last we remove all cyclones having identifier smaller than 70 since they do not represent real events and we select the cyclone we are interested in.

### Forecast Data

In [14]:
start_date = datetime(2023, 9, 7, 0, 0)

# Load cyclone dataframe with Mean sea level pressure value
df_storms_forecast = pdbufr.read_bufr(f"data/tracks/{start_date.strftime('%Y%m%d')}.bufr",
    columns=("stormIdentifier", "longStormName", "ensembleMemberNumber", "year", "month", "day", "hour", "latitude", "longitude", 
             "pressureReducedToMeanSeaLevel"))

# Load cyclone dataframe with Wind speed at 10m value
df1 = pdbufr.read_bufr(f"data/tracks/{start_date.strftime('%Y%m%d')}.bufr",
    columns=("stormIdentifier", "longStormName", "ensembleMemberNumber", "latitude", "longitude",
                "windSpeedAt10M"))

# Build the timeperiod column for the dataframe
timeperiod = []
start_date = datetime(df_storms_forecast.year[0], df_storms_forecast.month[0], df_storms_forecast.day[0], df_storms_forecast.hour[0])
for cyclone in df_storms_forecast.stormIdentifier.unique():
    df_cyclone = df_storms_forecast[df_storms_forecast.stormIdentifier == cyclone]
    df_cyclone.reset_index(inplace=True, drop=True)
    members = df_cyclone.ensembleMemberNumber.unique()
    for member in members:
        df_track = df_cyclone[df_cyclone.ensembleMemberNumber == member]
        for i in range(len(df_track)):
            timeperiod.append(6 * (i+1))

# Add the Wind speed at 10m column adn the timePeriod column to the storms dataframe 
df_storms_forecast["windSpeedAt10M"] = df1.windSpeedAt10M
df_storms_forecast["timePeriod"] = timeperiod
del df1

# Storms with stormIdentifier higher than 70 are not real storms
drop_condition = df_storms_forecast.stormIdentifier < '70'
df_storms_forecast = df_storms_forecast[drop_condition]

# Select the data of storm LEE (Storm Identifier: 13L)
df_storm_forecast = df_storms_forecast[df_storms_forecast.stormIdentifier == '13L']

### Observed data

In [9]:
# Load observed data
df_storms_observed = pd.read_csv('data/tracks/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

# Select the data of storm LEE
df_storm_observed = df_storms_observed[df_storms_observed.NAME.squeeze() == 'LEE']

## Tracks Location Computation

To plot the tracks in an interactive map we use the python library **ipyleaflet**. This library requires to have the locations saved in a list to represent plot them as polylines on a map. The following code converts the different tracks locations from a dataframe column to a list of locations lists in case of the ensemble tracks and to a locations list in case of the observed track. 

The average forecast track is also computed using the appropriate trigonometric formula. 

For each track we also compute the list of time steps, mean sea level pressure and wind speed for the correspondent location. 

### Ensemble Forecast Tracks

#### Ensemble Members to Plot Widget
This widget allows you to decide which ensembles you want to plot in the Map. Since several polylines in the Map makes it unstable, if you select more than 5 ensemble members it will not report all the rest of the information mentioned before. 

In [15]:
# Define the widget to select which ensemble members to plot
members = df_storm_forecast.ensembleMemberNumber.unique()
ens_members = widgets.SelectMultiple(
    options=members,
    value=[],
    description="Ensemble members to plot:",
    disable=False
)

ens_members.style.description_width = '168px'
ens_members.style.font_weight = 'bold'

# Widget for instruction on how to select the ensemble members to plot
message_ens = widgets.HTML(
    value="Multiple values can be selected with <b>shift</b> and/or <b>ctrl</b> (or <b>command</b>)",
)

display(message_ens)
display(ens_members)

HTML(value='Multiple values can be selected with <b>shift</b> and/or <b>ctrl</b> (or <b>command</b>)')

SelectMultiple(description='Ensemble members to plot:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

#### Tracks

In [ ]:
# Select the tracks in the forecast dataframe according to the ensemble mebers seleced by user
df_f = df_storm_forecast[df_storm_forecast.ensembleMemberNumber.isin(ens_members)]
df_f.reset_index(drop=True, inplace=True)

# List all the ensemble members tracks for the cyclone forecast and define the empty lists
members = df_f.ensembleMemberNumber.unique()
locations_f = []
timesteps_f = []
pressures_f = []
wind_speeds_f = []

# Cycle through the members of the forecast
for member in members:
    df_track = df_f[df_f.ensembleMemberNumber == member]
    df_track.reset_index(drop=True, inplace=True)

    # Create a date column in dataframe
    start = datetime(df_track.iloc[0]['year'], df_track.iloc[0]['month'], df_track.iloc[0]['day'], df_track.iloc[0]['hour'])
    df_track['date'] = start + timedelta(hours=6) * (df_track.index+1)
    df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)

    # Save the information from a single track in lists
    latitude = df_track.latitude.tolist()
    longitude = df_track.longitude.tolist()
    dates = df_track.date.tolist()
    pressure = df_track.pressureReducedToMeanSeaLevel.tolist()
    wind_speed = df_track.windSpeedAt10M.tolist()
    locs = []
    tmtstps = []
    press = []

    # Cycle through the list of location
    for i in range(len(latitude)):
        loc = (latitude[i], longitude[i])
        locs.append(loc)
        tmtstps.append(dates[i].strftime("%d-%m-%Y %H:%M"))
        press.append(pressure[i] * 10**-2) # Pa to hPa

    locations_f.append(locs)
    timesteps_f.append(tmtstps)
    pressures_f.append(press)
    wind_speeds_f.append(wind_speed)

### Observed Track

In [ ]:
# Convert the pandas dataframe columns to lists
latitude = df_storm_observed.LAT.squeeze().tolist()
longitude = df_storm_observed.LON.squeeze().tolist()
dates = df_storm_observed.ISO_TIME.squeeze().tolist()
locations_o = []
timesteps_o = []

# Cycle through the lists
for i in range(len(latitude)):
    loc = (latitude[i], longitude[i])
    date = datetime.strptime(dates[i][:-3], ('%Y-%m-%d %H:%M'))
    locations_o.append(loc)
    timesteps_o.append(date.strftime('%d-%m-%Y %H:%M'))

### Average Forecast Track

In [12]:
def meanposit(knpf, rlatpf, rlonpf):
    """
    Computes the average coordinates from the ensemble tracks at a specific timestep.
    
    knpf: int
        Number of ensemble tracks 
    rlatpf: numpy array
        Array containing the latitudes of the ensemble tracks
    rlonpf: numpy array
        Array containing the longitudes of the ensemble tracks
    
    Returns:
    rlatmean: float
        Mean latitude at the timestep
    rlonmean: float
        Mean longitude at the timestep
    """
    rpi = math.acos(0.0)
    rnomin = 0.0
    rdenom = 0.0
    rphisum = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * rlon
        rdenom += rcosphi
        rphisum += rlat
    
    rlabda = rnomin / rdenom
    rlonmean = rlabda * 180.0 / rpi
    
    rnomin = 0.0
    repsilon = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * (rlabda - rlon) ** 2
    
    repsilon = rnomin / (2.0 * knpf)
    rphimean = rphisum / float(knpf)
    rphi = rphimean + repsilon * math.sin(rphimean)
    rlatmean = rphi * 180.0 / rpi
    
    return rlatmean, rlonmean

In [ ]:
# Create empty dataframes to fill with each ensemble members information
members = df_storm_forecast.ensembleMemberNumber.unique()
df_lat_tracks = pd.DataFrame()
df_lon_tracks = pd.DataFrame()
df_prs_tracks = pd.DataFrame()
df_wds_tracks = pd.DataFrame()

# Cycle thorugh the ensemble members and save the information in the just created dataframes
for member in members:
    df_track = df_storm_forecast[df_storm_forecast.ensembleMemberNumber == member]
    df_track.reset_index(drop=True, inplace=True)
    df_lat_tracks[f'latitude{member}'] = df_track.latitude
    df_lon_tracks[f'longitude{member}'] = df_track.longitude
    df_prs_tracks[f'pressure{member}'] = df_track.pressureReducedToMeanSeaLevel
    df_wds_tracks[f'wind{member}'] = df_track.windSpeedAt10M

# Add date information for the average track
df_track.reset_index(drop=True, inplace=True)
start = datetime(df_track.iloc[0]['year'], df_track.iloc[0]['month'], df_track.iloc[0]['day'], df_track.iloc[0]['hour'])
dates = start + timedelta(hours=6) * (df_track.index+1)

# Cycle through the rows of df_lat_track and df_lon_tracks to compute the average track lat,lon and pressure and wind speed perecentiles
locations_avg = []
timesteps_avg = []
pressures_avg = []
wind_speeds_avg = []

for t in range(len(df_lat_tracks)):
    lat = df_lat_tracks.iloc[t].dropna().to_numpy()
    lon = df_lon_tracks.iloc[t].dropna().to_numpy()
    prs = df_prs_tracks.iloc[t].dropna().to_numpy()  * 10**-2 # Pa to hPa
    wds = df_wds_tracks.iloc[t].dropna().to_numpy()
    date = dates[t].strftime("%d-%m-%Y %H:%M")
    if len(lat) > 0:
        mean_lat_lon = meanposit(len(lat), lat, lon)
        pressures_avg.append(np.percentile(prs, [10, 25, 50, 75, 90]))
        wind_speeds_avg.append(np.percentile(wds, [10, 25, 50, 75, 90]))
        locations_avg.append(mean_lat_lon)
        timesteps_avg.append(date)

## Strike Probability Map Computation

The strike probability is the probbaility that a tropical cyclone will pass within a 300 km radius from a given location and within a time window of 48 hours. This information provides a quick assessment of high-risk areas. If you want to know more about check the following  __[link](https://charts.ecmwf.int/products/medium-tc-genesis?base_time=202309140000&layer_name=genesis_ts&projection=opencharts_global&valid_time=202309170000)__.

The strike probability map is computed through a series of operations relying on the construction of a space-partitioning data structure called *k*-d tree. Since the code to compute the strike probbaility map is quite long, we preffered to not report it here in the tutorial notebook. If you are interesed in the computation of the strike probability map you can chek out the python script *strike_map.py*. 

The strike probability map is saved to a raster file to visualize it in the ipyleaflet map. 

In [ ]:
from strike_map import strike_probability_map

# Compute the strike probability map and save it to raster file
df_storm = df_storm_forecast.copy()
strike_map_xr, tif_path = strike_probability_map(df_storm)

## Plot Data in an Interactive Map

### Forecast Ensemble Tracks

In [ ]:
# Define colours list for the ensemble tracks
colours = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "brown"]
tracks_layer = []
colour = 0
i = 0

# Cycle on the ensembles of the forecast track
for locs in locations_f:

    tmtstps = timesteps_f[i]
    press = pressures_f[i]
    wind = wind_speeds_f[i]
    
    # Define the ensemble track polyline for the map
    track = ipyleaflet.Polyline(
        locations=locs,
        color=colours[colour],
        fill=False,
        weight=2,
    )

    # If the number of ensemble members is less than 5, define the markers for each position of the cyclone ensemble forecast
    if len(locations_f) <= 5:
        markers = []
        for j in range(len(locs)):
            marker = ipyleaflet.CircleMarker(
                location=locs[j],
                radius=1,
                color=colours[colour],
                popup=widgets.HTML(value=f'<center><b>{tmtstps[j]}</b> <br> Pressure: {press[j]:.2f} hPa <br> Wind speed: {wind[j]:.2f} m/s</center>')
            )
            markers.append(marker)
        markers_layer = ipyleaflet.LayerGroup(layers=markers)
        track_layer = ipyleaflet.LayerGroup(layers=[track, markers_layer])
        tracks_layer.append(track_layer)
    else:
        tracks_layer.append(track)
        
    colour += 1
    if colour == len(colours):
        colour = 0
    
    i += 1

### Average Forecast Track

In [ ]:
# Define average forecast track polyline for the map
track_avg = ipyleaflet.Polyline(
        locations=locations_avg,
        color="black",
        fill=False,
        weight=3,
    )

# Define the markers element for each position of the average track
marker_avg = []
for avg in range(len(locations_avg)):
    marker = ipyleaflet.CircleMarker(
        location = locations_avg[avg],
        radius=1,
        color="black",
        popup=widgets.HTML(value=f"<center><b> {timesteps_avg[avg]} </b> </center>"
                            f"Percentiles: Pressure || Wind speed <br>"
                            f"10-th: {pressures_avg[avg][0]:.1f} hPa || {wind_speeds_avg[avg][0]:.2f} m/s <br>"
                            f"25-th: {pressures_avg[avg][1]:.1f} hPa || {wind_speeds_avg[avg][1]:.2f} m/s <br>"
                            f"50-th: {pressures_avg[avg][2]:.1f} hPa || {wind_speeds_avg[avg][2]:.2f} m/s <br>"
                            f"75-th: {pressures_avg[avg][3]:.1f} hPa || {wind_speeds_avg[avg][3]:.2f} m/s <br>"
                            f"90-th: {pressures_avg[avg][4]:.1f} hPa || {wind_speeds_avg[avg][4]:.2f} m/s"
                            )
    )
    marker_avg.append(marker)

### Observed Track

In [ ]:
# Define observed tracks polyline element for the map
track_o = ipyleaflet.Polyline(
        locations=locations_o,
        color= "#ff00ff",
        fill=False,
        weight=2,
    )

# Define the markers element for each position of the observed track
marker_o = []
for o in range(len(locations_o)):
    marker = ipyleaflet.CircleMarker(
        location = locations_o[o],
        radius=1,
        color="#ff00ff",
        popup=widgets.HTML(value=f'<b> {timesteps_o[o]} </b>')
    )
    marker_o.append(marker)

### Strike Probability Map

In [ ]:
# Load raster file and define layer for strike probability map
client = TileClient(tif_path)
palette = ["#8df52c", "#6ae24c", "#61bb30", "#508b15", "#057941", "#2397d1", "#557ff3", "#143cdc", "#3910b4", "#1e0063"]
stp_map = get_leaflet_tile_layer(client, name = "Strike Probability Map", opacity = 0.8, palette = palette, nodata=0.0)

# Define colormap for the strike probability map
with rasterio.open(tif_path) as r:
    minv = "%.2f" % round(r.read(1).ravel().min(), 1)
    maxv = "%.2f" % round(r.read(1).ravel().max(), 1)

cmap_control = ipyleaflet.ColormapControl(
                            caption = "Strike probability",
                            colormap = bc.StepColormap(palette),
                            value_min = float(minv),
                            value_max = float(maxv),
                            position = 'topright',
                            transparent_bg = True
                            )

### Map and Layers Definition

In [ ]:
# Define the map
initial_lat_lon = (df_storm_forecast.latitude.iloc[0], df_storm_forecast.longitude.iloc[0])
tc_track_map = ipyleaflet.Map(
        center=initial_lat_lon,
        basemap=ipyleaflet.basemaps.OpenStreetMap.France,
        zoom = 3.0,
        # scroll_wheel_zoom=True,
    )

# Add the observed track layer to the map 
markers_layer_o = ipyleaflet.LayerGroup(layers=marker_o)
layer_group_o = ipyleaflet.LayerGroup(layers=[track_o, markers_layer_o], name='Observed Track')
tc_track_map.add_layer(layer_group_o)

# Add the ensemble tracks layer to the map
tracks_layer_group = ipyleaflet.LayerGroup(layers=tracks_layer, name='Forecasted Ensemble Tracks')
tc_track_map.add_layer(tracks_layer_group)

# Add the average forecast track layer to the map
markers_layer_avg = ipyleaflet.LayerGroup(layers=marker_avg)
layer_group_avg = ipyleaflet.LayerGroup(layers=[track_avg, markers_layer_avg], name='Average Forecast Track')
tc_track_map.add_layer(layer_group_avg)

# Add the strike probability map layer to the map
tc_track_map.add_layer(stp_map)
tc_track_map.add_control(cmap_control)

# Add layers widget to the map
layers_control = ipyleaflet.LayersControl()
tc_track_map.add_control(layers_control)
tc_track_map.add_control(ipyleaflet.FullScreenControl());

# Display the map
display(tc_track_map)